In [3]:
import os
import json

import numpy as np
import pandas


from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, InputLayer, LeakyReLU, Reshape, Flatten
from keras.layers.pooling import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D

from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam

import librosa

Using TensorFlow backend.


In [4]:
def load_index(fold_id, key='train', n=None):
    
    with open(os.path.join(DESTDIR, 'experiment_{:02d}.json'.format(fold_id)), 'r') as fdesc:
        index = json.load(fdesc)[key]
    
    perm = np.random.permutation(len(index))
    if n is not None:
        perm = perm[:n]
        
    index = [index[_] for _ in perm]
    
    return index

In [5]:
def preprocess(X):
    return librosa.logamplitude(X**2, ref_power=np.max, top_db=80)


def load_data(index, n_harm=1):
    
    X, Y = [], []
    for item in index:
        with np.load(os.path.join(DESTDIR, 'features', '{}.npz'.format(item['filename']))) as data:
            if n_harm is None:
                n_harm = data['C'].shape[0]
            X.append(preprocess(data['C'][:n_harm]))
            Y.append(item['classID'])
    
    X = np.asarray(X).swapaxes(3, 1)
    Y = np.asarray(Y)
    
    return X, Y

In [6]:
DESTDIR = '/home/bmcfee/working/UrbanSound8K/'

In [7]:
FOLD_ID = 1

In [8]:
index = load_index(FOLD_ID)

In [9]:
test_index = load_index(FOLD_ID, key='test')

In [51]:
N_HARM = 4

In [52]:
X, Y = load_data(index, n_harm=N_HARM)

In [53]:
Xt, Yt = load_data(test_index, n_harm=N_HARM)

In [54]:
model = Sequential()

model.add(InputLayer(input_shape=X.shape[1:], name='input'))
model.add(BatchNormalization())

# 9 bins = +- 1 semitone
# 4 * 9 bins = 1 octave span
model.add(Conv2D(8, 5, (4 - X.shape[3] + 1) * 9, bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(16, 5, 5,  bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

#model.add(MaxPooling2D(pool_size=(5, 5)))

#model.add(Conv2D(16, 3, model.output_shape[2], bias=False))
#model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1))


#model.add(AveragePooling2D(pool_size=(model.output_shape[1], 1)))
model.add(MaxPooling2D(pool_size=(model.output_shape[1], 1)))
model.add(Flatten())

model.add(Dense(10, activation='softmax'))



In [55]:
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [56]:
weight_fn = '/home/bmcfee/working/UrbanSound8K/models_{}.hdf5'.format(N_HARM)

In [57]:
history = model.fit(X, to_categorical(Y, nb_classes=10),
                    callbacks=[EarlyStopping('val_loss', patience=10),
                               ModelCheckpoint(weight_fn, save_best_only=True)],
                    validation_split=0.25, batch_size=32, shuffle=True,
                    nb_epoch=30)

Train on 5894 samples, validate on 1965 samples
Epoch 1/30
5894/5894 [==============================] - 24s - loss: 1.6565 - acc: 0.4141 - val_loss: 1.4046 - val_acc: 0.5053
Epoch 2/30
5894/5894 [==============================] - 24s - loss: 1.0514 - acc: 0.6469 - val_loss: 1.0905 - val_acc: 0.5995
Epoch 3/30
5894/5894 [==============================] - 24s - loss: 0.8227 - acc: 0.7346 - val_loss: 1.0001 - val_acc: 0.6814
Epoch 4/30
5894/5894 [==============================] - 24s - loss: 0.7003 - acc: 0.7749 - val_loss: 0.7753 - val_acc: 0.7369
Epoch 5/30
5894/5894 [==============================] - 24s - loss: 0.6186 - acc: 0.8071 - val_loss: 0.7330 - val_acc: 0.7532
Epoch 6/30
5894/5894 [==============================] - 24s - loss: 0.5456 - acc: 0.8281 - val_loss: 0.6828 - val_acc: 0.7684
Epoch 7/30
5894/5894 [==============================] - 23s - loss: 0.5109 - acc: 0.8359 - val_loss: 0.6950 - val_acc: 0.7710
Epoch 8/30
5894/5894 [==============================] - 24s - loss: 0.

In [58]:
model.load_weights(weight_fn)

In [31]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

1
864/873 [============================>.] - ETA: 0s

[1.3198909005758279, 0.71363115699840163]

In [40]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

2
873/873 [==============================] - 1s     


[0.95194993129710559, 0.71363115706667735]

In [50]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

3
873/873 [==============================] - 1s     


[1.2872762315461725, 0.68155784664285146]

In [60]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

4
873/873 [==============================] - 1s     


[1.1552962344673219, 0.69072164975763872]